In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import pandas as pd
import cv2
import fnmatch
import os
import glob

In [ ]:
df = pd.read_csv('data.txt', sep=" ", header=None)
df.columns = ["ID", "background_tissue", "class",
              "severity", "x", "y", "rad"]
df.index = df.index + 1
df = df.dropna()

In [ ]:
im = Image.open('all-mias/mdb001.pgm')

In [ ]:
im.getpixel( (535,425) )

155

In [ ]:
last = 'start'

In [ ]:
for index, row in df.iterrows():
    path = 'all-mias/{name}.pgm'.format(name=row['ID'])
    if last != path:
        im = Image.open(path).convert("RGB")
    x = int(row['x'])
    y = int(row['y'])
    r = int(row['rad'])
    c = row['severity']
    img_draw(im, x, 1024 - y, r, c)
    im.save("marked/{}.png".format(row['ID']))
    last = path

In [ ]:
df.head(1)

,ID,background_tissue,class,severity,x,y,rad
1,mdb001,G,CIRC,B,535,425,197.0


In [ ]:
im = cv2.imread('all-mias/mdb001.pgm')

In [ ]:
h,w,d = im.shape

In [ ]:
mask = np.zeros((h,w), np.uint8)

In [ ]:
cv2.circle(mask,(535,1024-425),197,1,thickness=-1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
masked_data = cv2.bitwise_and(im, im, mask=mask)

In [ ]:
cv2.imshow("masked", masked_data)
cv2.waitKey(0)

-1

In [ ]:
cv2.mean(im, mask = mask)

(150.78105081826013, 150.78105081826013, 150.78105081826013, 0.0)

In [ ]:
np.pow(cv2.meanStdDev(im, mask = mask)[1])

array([[7.52443877],
       [7.52443877],
       [7.52443877]])

In [ ]:
cv2.imwrite("masked.png", masked_data)

True

In [ ]:
def get_ave():
    